In [1]:
!pip list

Package         Version
--------------- -------
joblib          1.1.0
numpy           1.21.4
pandas          1.3.4
pip             21.3.1
python-dateutil 2.8.2
pytz            2021.3
scikit-learn    1.0.1
scipy           1.7.2
setuptools      58.3.0
six             1.16.0
threadpoolctl   3.0.0
wheel           0.37.0
